In [1]:
import h5py
import numpy as np

In [2]:
from astropy.io import fits

DES-Y3

In [5]:
with h5py.File('/pscratch/sd/j/jjeffers/temp/TXPipe/data/desy3/FINALRUN/binned_shear_catalog.hdf5','r') as f:
    print(np.average(f['shear/bin_all/g1'][:],weights=f['shear/bin_all/weight'][:]))
    print(np.average(f['shear/bin_all/g2'][:],weights=f['shear/bin_all/weight'][:]))
    print(len(f['shear/bin_all/g2'][:]))

-7.0073987065087775e-12
1.1986475849103899e-12
100204026


KiDS-1000

In [9]:
with h5py.File('/pscratch/sd/j/jjeffers/temp/TXPipe/data/kids/FINALRUN/binned_shear_catalog.hdf5','r') as f:
    Nmask = f['shear/bin_all/dec'][:] > -25.0
    Smask = f['shear/bin_all/dec'][:] < -25.0

    print(np.average(f['shear/bin_all/g1'][:][Nmask],weights=f['shear/bin_all/weight'][:][Nmask]))
    print(np.average(f['shear/bin_all/g2'][:][Nmask],weights=f['shear/bin_all/weight'][:][Nmask]))
    print(len(f['shear/bin_all/g2'][:][Nmask]))
    
    print(np.average(f['shear/bin_all/g1'][:][Smask],weights=f['shear/bin_all/weight'][:][Smask]))
    print(np.average(f['shear/bin_all/g2'][:][Smask],weights=f['shear/bin_all/weight'][:][Smask]))
    print(len(f['shear/bin_all/g2'][:][Smask]))

-9.52258767442577e-10
1.0358985455429282e-09
9863348
-1.0475637139720932e-09
2.4188100940327516e-10
11298657


HSC-Y3


In [20]:
def rd2tp(ra,dec):
    """Convert ra,dec -> tht,phi"""
    tht = (-dec+90.0)/180.0*np.pi
    phi = ra/180.0*np.pi
    return tht,phi

def tp2rd(tht,phi):
    """Convert tht,phi -> ra,dec"""
    ra  = phi/np.pi*180.0
    dec = -1*(tht/np.pi*180.0-90.0)
    return ra,dec

def pointpick(ra,dec,mask,g1,g2,w):
    nside        = hp.npix2nside(mask.shape[0])
    tht,phi = rd2tp(ra,dec)
    pix = hp.ang2pix(nside,tht,phi)
    idx = np.where(mask[pix]>0)[0]
    #ra,dec = tp2rd(tht[idx],phi[idx]) 
    g1 = g1[idx]
    g2 = g2[idx]
    w = w[idx]
    return g1,g2,w

In [21]:
import healpy as hp

In [22]:
fields = ['GAMA09H','GAMA15H','HECTOMAP','WIDE12H','VVDS','XMM']
path = '/global/cfs/cdirs/lsst/groups/WL/projects/txpipe-sys-tests/hsc-y3/shear/'

for field in fields:
    print(f'{field}:')
    #make mask for the subfield:
    g = fits.open(path+f'{field}_calibrated.fits')
    tht,phi = rd2tp(np.asarray(g[1].data['i_ra']), np.asarray(g[1].data['i_dec']))
    pix     = hp.ang2pix(1024,tht,phi)
    mask= np.bincount(pix,minlength=hp.nside2npix(1024))
    mask[mask>0]=1
    #open precalibrated shear catalog
    with h5py.File(path+'txpipe_allfield_shear.h5','r') as h:
        #mask the data and find the objects within the subfield
        g1,g2,weight = pointpick(h['shear/ra'][:],h['shear/dec'][:],mask,h['shear/g1'][:],h['shear/g2'][:],h['shear/weight'][:])
        print(np.average(g1,weights=weight))
        print(np.average(g2,weights=weight))
        print(len(g1))

GAMA09H:
-0.0012003017742054437
0.0010092385147866013
4866360
GAMA15H:
-0.0007990328855668942
-0.0012249324414537197
2573993
HECTOMAP:
-0.0018714264488715524
-0.0007864205066788969
2650656
WIDE12H:
0.00118017920431754
0.00023965347279769674
7279084
VVDS:
3.689602068764318e-05
-0.0009752990378493198
6015472
XMM:
-0.0015366043247956838
-0.0011481756876606723
1875312
